Imports

In [ ]:
#Imports
import logging
import json
import pandas as pd

from medcat.cdb import CDB
from medcat.config_rel_cat import ConfigRelCAT
from medcat.rel_cat import RelCAT
from medcat.utils.relation_extraction.base_component import BaseComponent_RelationExtraction
from medcat.utils.relation_extraction.bert.model import BaseModel_RelationExtraction
from medcat.utils.relation_extraction.bert.config import BaseConfig_RelationExtraction
from medcat.utils.relation_extraction.tokenizer import BaseTokenizerWrapper_RelationExtraction

import sys, os
utils_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'utils'))
if utils_path not in sys.path:
    sys.path.insert(0, utils_path)

from general_utils import load_data

Data Loading

In [ ]:
# Load data
df = load_data("../data/training_dataset.csv")
print(f"Loaded {len(df)} records")

In [ ]:
#Inspect df
df.head()

Create Training Dataset

In [ ]:
#Insert marker function
def insert_marker(txt, start, end, tag_open, tag_close):
                    return txt[:start] + tag_open + txt[start:end] + tag_close + txt[end:]

In [ ]:
# Create dataset for training
rows = []

for _, row in df.iterrows():
    doc_id = row["doc_id"]
    text = row["note_text"]
    
    # Parse the JSON columns - handle both string and already-parsed cases
    entities = row["entities_json"] if isinstance(row["entities_json"], list) else json.loads(row["entities_json"])
    dates = row["dates_json"] if isinstance(row["dates_json"], list) else json.loads(row["dates_json"])
    relative_dates = row["relative_dates_json"] if isinstance(row["relative_dates_json"], list) else json.loads(row["relative_dates_json"]) if "relative_dates_json" in row else []
    
    # Combine absolute and relative dates
    all_dates = dates + relative_dates
    
    # Get original relations for labeling
    relations = row["relations_json"] if isinstance(row["relations_json"], list) else json.loads(row["relations_json"])
    
    # Build relation pairs from validated relations - match by IDs
    relation_pairs = {tuple(sorted([str(L["date_id"]), str(L["entity_id"])])) for L in relations}
    
    # Create pairs for all date-entity combinations
    for date in all_dates:
        for entity in entities:
            # Use IDs for matching
            date_id = str(date["id"])
            entity_id = str(entity["id"])
            
            # Determine label
            if tuple(sorted([date_id, entity_id])) in relation_pairs:
                label, label_id = "RELATION", 1
            else:
                label, label_id = "NO_RELATION", 0
            
            # Insert markers
            s1, e1 = date.get("start"), date.get("end")
            s2, e2 = entity.get("start"), entity.get("end")
            
            if s1 is not None and s2 is not None:
                if s1 < s2:
                    marked = insert_marker(text, s2, e2, "[s2]", "[e2]")
                    marked = insert_marker(marked, s1, e1, "[s1]", "[e1]")
                else:
                    marked = insert_marker(text, s1, e1, "[s2]", "[e2]")
                    marked = insert_marker(marked, s2, e2, "[s1]", "[e1]")
            else:
                marked = text
            
            rows.append({
                "relation_token_span_ids": None,
                "ent1_ent2_start": (s1, s2),
                "ent1": date.get("value", ""),
                "ent2": entity.get("value", ""),
                "label": label,
                "label_id": label_id,
                "ent1_type": "DATE",
                "ent2_type": "ENTITY",
                "ent1_id": date_id,
                "ent2_id": entity_id,
                "ent1_cui": None,
                "ent2_cui": None,
                "doc_id": doc_id,
                "text": marked
            })

training_df = pd.DataFrame(rows)
print(f"Created {len(training_df)} date-entity pairs (including relative dates)")

In [ ]:
#Inspect data
print(f"Dataset: {len(training_df)} samples")
print("\nLabel distribution:")
print(training_df["label"].value_counts())

training_df.head()

In [ ]:
# Per-document summary from original data
summary = []

for _, row in df.iterrows():
    ## Parse the JSON columns - handle both string and already-parsed cases
    entities = row["entities_json"] if isinstance(row["entities_json"], list) else json.loads(row["entities_json"])
    dates = row["dates_json"] if isinstance(row["dates_json"], list) else json.loads(row["dates_json"])
    relative_dates = row["relative_dates_json"] if isinstance(row["relative_dates_json"], list) else json.loads(row["relative_dates_json"]) if "relative_dates_json" in row else []
    relations = row["relations_json"] if isinstance(row["relations_json"], list) else json.loads(row["relations_json"])
    
    summary.append({
        "doc_id": row["doc_id"],
        "n_entities": len(entities),
        "n_dates": len(dates),
        "n_relative_dates": len(relative_dates),
        "n_relations": len(relations),
    })

doc_level = pd.DataFrame(summary)
doc_level

In [ ]:
# Pair-based stats from the generated df
pair_stats = (
    training_df.groupby("doc_id")
      .agg(
          total_pairs=("label", "size"),
          relation_pairs=("label", lambda s: (s == "RELATION").sum()),
      )
      .reset_index()
)

pair_stats["relation_pct"] = (100 * pair_stats["relation_pairs"] / pair_stats["total_pairs"]).round(1)
pair_stats

In [ ]:
# Merge stats with doc_level summary
doc_level = (
    doc_level.merge(pair_stats[["doc_id", "total_pairs", "relation_pct"]], on="doc_id", how="left")
             .sort_values("doc_id")
             .reset_index(drop=True)
)

# Calculate additional metrics
total_pairs_overall = len(training_df)
relation_total = (training_df["label"] == "RELATION").sum()
relation_pct_overall = 100 * (training_df["label"] == "RELATION").mean()

print(f"total number of date-entity pairs is {total_pairs_overall}")
print(f"total number of relations: {relation_total}")
print(f"percentage positive class: {relation_pct_overall:.1f}%")

#Look at overall doc level summary
doc_level

In [ ]:
# Save training dataset
training_df.to_csv("../data/relcat_training_data.tsv", sep="\t", index=False)

Training & Model Config

In [ ]:
#Choose model to use - any BERT model from HuggingFace can be used, see: https://huggingface.co/google-bert
model = "bert-base-uncased"

In [ ]:
#Set path to save trained model and checkpoints to
model_save_path = '../models/relcat_models'

In [ ]:
#Create RelCAT config and set parameters
config = ConfigRelCAT()
config.general.log_level = logging.INFO
config.general.model_name = model
#logging.basicConfig(level=logging.INFO)

In [ ]:
#Hidden size, model size and hidden layers
config.model.hidden_size= 256
config.model.model_size = 2304 # 4096 for llama

In [ ]:
# Further config
config.general.cntx_left = 15
config.general.cntx_right = 15
config.general.window_size = 400
config.train.nclasses = 2
config.train.nepochs = 3
config.model.freeze_layers = False
config.general.limit_samples_per_class = 300
config.train.batch_size = 32
config.train.lr = 3e-5
config.train.adam_epsilon = 1e-8
config.train.adam_weight_decay = 0.0005
config.train.class_weights = [0.3027, 1.6973]
config.train.enable_class_weights = True

In [ ]:
#Create CDB
cdb = CDB()

In [ ]:
#Create tokenizer
tokenizer = BaseTokenizerWrapper_RelationExtraction.load(tokenizer_path=config.general.model_name,
                                                       relcat_config=config)

In [ ]:
#Add special tokens
special_ent_tokens = ["[s1]", "[e1]", "[s2]", "[e2]"]
tokenizer.hf_tokenizers.add_tokens(special_ent_tokens, special_tokens=True)
tokenizer.hf_tokenizers.add_special_tokens({'pad_token': '[PAD]'}) # used in llama tokenizer

In [ ]:
#Add tokens to config
config.general.tokenizer_relation_annotation_special_tokens_tags = special_ent_tokens
config.general.annotation_schema_tag_ids = tokenizer.hf_tokenizers.convert_tokens_to_ids(special_ent_tokens)

In [ ]:
#Create RelCAT object
relCAT = RelCAT(cdb, config=config)

In [ ]:
# Load model configuration
model_config = BaseConfig_RelationExtraction.load(pretrained_model_name_or_path=config.general.model_name,
                                                 relcat_config=config)

In [ ]:
# Update vocab size in model config to match tokenizer
model_config.hf_model_config.vocab_size = tokenizer.get_size()

In [ ]:
# set the padding idx in the model config and relcat config, this is necesasry as it depends on what tokenizer you use
config.model.padding_idx = model_config.pad_token_id = tokenizer.get_pad_id()

In [ ]:
# Load model
model = BaseModel_RelationExtraction.load(pretrained_model_name_or_path=config.general.model_name,
                                         model_config=model_config,
                                         relcat_config=config)

In [ ]:
# Resize embeddings to match tokenizer
model.hf_model.resize_token_embeddings(len(tokenizer.hf_tokenizers))

In [ ]:
# Create RelCAT component
component = BaseComponent_RelationExtraction(tokenizer=tokenizer, config=config)
component.model = model
component.model_config = model_config
component.relcat_config = config
component.tokenizer = tokenizer
relCAT.component = component

Training

In [ ]:
# Train the model using the dataset we created earlier
relCAT.train(
    train_csv_path="../data/relcat_training_data.tsv",  
    checkpoint_path=model_save_path
)

In [ ]:
#Save model
relCAT.save(save_path=model_save_path)